In [1]:
import numpy as np

# Keras
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

%matplotlib inline

In [2]:
# returns train, inference_encoder and inference_decoder models
def models(n_input, n_output, n_units):
    """
    Gets and prints the spreadsheet's header columns

    Parameters
    ----------
    n_input : str
        Length of input sequence, e.g number of integers
    n_output : bool, optional
        Length of output sequence, e.g number of integers
    n_units : bool, optional
        N umber of cells to create in the encoder and decoder models

    Returns
    -------
    model, encoder_model, decoder_model
        3 models, 
    """
    
    # training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    
    # training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)

    # inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    return model, encoder_model, decoder_model

In [3]:
# generate a sequence of random integers
def generate_sequence(length, max_number):
    return np.random.randint(max_number, size=length)

In [16]:
# prepare data for the LSTM
def generate_dataset(n_in, max_number, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, max_number)
        # define target sequence
        target = sorted(source)
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        src_encoded = to_categorical(source, num_classes=max_number)
        tar_encoded = to_categorical(target, num_classes=max_number)
        tar2_encoded = to_categorical(target_in, num_classes=max_number)
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)

    return np.array(X1), np.array(X2), np.array(y)

In [17]:
# Apply encoder-decoder LSTM model

In [18]:
# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 6

In [19]:
# define model
train, encoder, decoder = models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
# generate training dataset
X1, X2, y = generate_dataset(n_steps_in, n_features, 100000)
print(X1.shape, X2.shape, y.shape)
print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

(100000, 6, 51) (100000, 6, 51) (100000, 6, 51)
X1=[49, 30, 27, 5, 45, 23], X2=[0, 5, 23, 27, 30, 45], y=[5, 23, 27, 30, 45, 49]


In [21]:
# train model
train.fit([X1, X2], y, epochs=1)

3125/3125 [==============================] - 33s 10ms/step - loss: 0.3918 - accuracy: 0.9119


In [27]:
# generate target given source sequence
def predict_sequence(encoder, decoder, source, n_steps, max_number):
    # encode
    state = encoder.predict(source)
    # start of sequence input
    target_seq = np.array([0.0 for _ in range(max_number)]).reshape(1, 1, max_number)
    
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = decoder.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return np.array(output)

In [28]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
    X1, X2, y = generate_dataset(n_steps_in, n_features, 1)
    target = predict_sequence(encoder, decoder, X1, n_steps_out, n_features)
    if np.array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
        correct += 1
print('Accuracy: %.3f%%' % (float(correct)/float(total)*100.0))

Accuracy: 99.00%


In [30]:
for _ in range(10):
	X1, X2, y = generate_dataset(n_steps_in, n_features, 1)
	target = predict_sequence(encoder, decoder, X1, n_steps_out, n_features)
	print('Input=%s target=%s, prediction=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

Input=[20, 14, 45, 9, 12, 19] target=[9, 12, 14, 19, 20, 45], prediction=[9, 12, 14, 19, 20, 45]
Input=[5, 45, 45, 22, 35, 48] target=[5, 22, 35, 45, 45, 48], prediction=[5, 22, 35, 45, 45, 48]
Input=[27, 13, 48, 0, 31, 44] target=[0, 13, 27, 31, 44, 48], prediction=[0, 13, 27, 31, 44, 48]
Input=[12, 43, 36, 31, 1, 9] target=[1, 9, 12, 31, 36, 43], prediction=[1, 9, 12, 31, 36, 43]
Input=[9, 48, 21, 48, 10, 15] target=[9, 10, 15, 21, 48, 48], prediction=[9, 10, 15, 21, 48, 48]
Input=[7, 36, 44, 10, 12, 24] target=[7, 10, 12, 24, 36, 44], prediction=[7, 10, 12, 24, 36, 44]
Input=[2, 46, 12, 1, 9, 31] target=[1, 2, 9, 12, 31, 46], prediction=[1, 2, 9, 12, 31, 46]
Input=[17, 36, 30, 12, 13, 18] target=[12, 13, 17, 18, 30, 36], prediction=[12, 13, 17, 18, 30, 36]
Input=[41, 9, 50, 30, 9, 4] target=[4, 9, 9, 30, 41, 50], prediction=[4, 9, 9, 30, 41, 50]
Input=[44, 50, 25, 14, 13, 47] target=[13, 14, 25, 44, 47, 50], prediction=[13, 14, 25, 44, 47, 50]
